In [14]:
import pandas as pd
import sqlite3
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer


In [16]:

# Download VADER lexicon if not already done
nltk.download('vader_lexicon')
print('vader lex downloaded')
# Initialize VADER sentiment analyzer
sid = SentimentIntensityAnalyzer()

# Connect to SQLite database
conn = sqlite3.connect(r"C:\Users\20230895\OneDrive - TU Eindhoven\Desktop\Data Science BsC\Year 1\Quartile 4\JBG030_DBL0\DBL0_easyjet\tweets4.db", timeout=30)
cursor = conn.cursor()
query = """ALTER table emb_tweetv2 ADD COLUMN if not exists table sent_vader REAL"""

cursor.execute(query)
# Define batch size
batch_size = 10000

def process_batch(batch):
    for row in batch:
        tweet_id = row[0]
        norm_tweet = row[1]
        # Analyze sentiment
        sentiment = sid.polarity_scores(norm_tweet)['compound']
        # Update the sentiment value in the database
        cursor.execute("UPDATE emb_tweet_v2 SET sent_vader = ? WHERE id_str = ?", (sentiment, tweet_id))

def main():
    # Fetch total number of rows in the table
    cursor.execute("SELECT COUNT(*) FROM emb_tweet_v2")
    total_rows = cursor.fetchone()[0]
    print(f"Total rows to process: {total_rows}")

    # Process in batches
    offset = 0
    while offset < total_rows:
        cursor.execute("SELECT id_str, norm_tweets FROM emb_tweet_v2 LIMIT ? OFFSET ?", (batch_size, offset))
        batch = cursor.fetchall()
        if not batch:
            break
        process_batch(batch)
        conn.commit()
        offset += batch_size
        print(f"Processed {offset} / {total_rows}")

    # Close the database connection
    conn.close()

if __name__ == "__main__":
    main()


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\20230895\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


vader lex downloaded
Total rows to process: 4413045
Processed 10000 / 4413045
Processed 20000 / 4413045
Processed 30000 / 4413045
Processed 40000 / 4413045
Processed 50000 / 4413045
Processed 60000 / 4413045
Processed 70000 / 4413045
Processed 80000 / 4413045
Processed 90000 / 4413045
Processed 100000 / 4413045
Processed 110000 / 4413045
Processed 120000 / 4413045
Processed 130000 / 4413045
Processed 140000 / 4413045
Processed 150000 / 4413045
Processed 160000 / 4413045
Processed 170000 / 4413045
Processed 180000 / 4413045
Processed 190000 / 4413045
Processed 200000 / 4413045
Processed 210000 / 4413045
Processed 220000 / 4413045
Processed 230000 / 4413045
Processed 240000 / 4413045
Processed 250000 / 4413045
Processed 260000 / 4413045
Processed 270000 / 4413045
Processed 280000 / 4413045
Processed 290000 / 4413045
Processed 300000 / 4413045
Processed 310000 / 4413045
Processed 320000 / 4413045
Processed 330000 / 4413045
Processed 340000 / 4413045
Processed 350000 / 4413045
Processed 36

In [ ]:
conn = sqlite3.connect(r"C:\Users\20230895\OneDrive - TU Eindhoven\Desktop\Data Science BsC\Year 1\Quartile 4\JBG030_DBL0\DBL0_easyjet\tweets4.db", timeout=30)
cursor = conn.cursor()

query = """ALTER table tweets ADD COLUMN if not exists table sent_vader REAL"""
cursor.execute(query)
conn = sqlite3.connect(db_file)
cursor = conn.cursor()

# Fetch the total number of rows to update
cursor.execute("SELECT COUNT(*) FROM tweets")
total_rows = cursor.fetchone()[0]

# Function to update a batch of tweets
def update_batch(offset, batch_size):
    update_query = """
    UPDATE tweets
    SET sent_vader = (
        SELECT et.sent_vader
        FROM emb_tweet_v2 et
        WHERE et.id_str = tweets.id_str
    )
    WHERE id_str IN (
        SELECT id_str
        FROM tweets
        LIMIT ? OFFSET ?
    )
    """
    cursor.execute(update_query, (batch_size, offset))
    conn.commit()

try:
    for offset in range(0, total_rows, batch_size):
        update_batch(offset, batch_size)
        print(f"Batch starting at offset {offset} updated successfully.")
        
except sqlite3.Error as e:
    print(f"An error occurred: {e}")
    conn.rollback()

finally:
    cursor.close()
    conn.close()



## Plot 1: stacked line chart

In [ ]:
# Connect to the SQLite database
conn = sqlite3.connect(r'C:\Users\20230895\OneDrive - TU Eindhoven\Desktop\Data Science BsC\Year 1\Quartile 4\JBG030_DBL0\DBL0_easyjet\tweets4.db')

query = """
SELECT tw.created_at, tw.text, tw.sent_vader
FROM tweets tw
JOIN extended_tweets ext ON tw.id_str = ext.id_str
WHERE (ext.full_text LIKE '%@easyJet%' AND tw.user_id_str != '38676903')
   OR (ext.full_text LIKE '%@Ryanair%' AND tw.user_id_str != '1542862735')
   OR (ext.full_text LIKE '%@British_Airways%' AND tw.user_id_str != '18332190')
   OR (ext.full_text LIKE '%@AmericanAir%' AND tw.user_id_str != '22536055')
   OR (ext.full_text LIKE '%@VirginAtlantic%' AND tw.user_id_str != '20626359')
"""

# Execute the query and fetch the data into a DataFrame
df = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()
df

In [ ]:
df1 = pd.DataFrame()
# Convert the created_at column to pandas datetime type
df1['created_at'] = pd.to_datetime(df['created_at'], format='%a %b %d %H:%M:%S %z %Y')

# Ensure that sent_vader contains only numeric values
df1['sent_vader'] = pd.to_numeric(df['sent_vader'], errors='coerce')

# Drop rows where sent_vader is NaN (due to non-numeric values)
df.dropna(subset=['sent_vader'], inplace=True)

# Define a function to extract the airline from the text
def extract_airline(text):
    if '@easyJet' in text:
        return 'easyJet'
    elif '@Ryanair' in text:
        return 'Ryanair'
    elif '@British_Airways' in text:
        return 'British_Airways'
#    elif '@AmericanAir' in text:
#        return 'AmericanAir'
    elif '@VirginAtlantic' in text:
        return 'VirginAtlantic'
    

# Apply the function to create a new column 'airline'
df1['airline'] = df['text'].apply(extract_airline)

# Resample the data to monthly frequency and calculate the average sentiment
df1.set_index('created_at', inplace=True)
monthly_avg_sentiment = df1.groupby('airline').resample('M').mean(numeric_only=True).reset_index()

# Pivot the data to have airlines as columns
pivot_df = monthly_avg_sentiment.pivot(index='created_at', columns='airline', values='sent_vader')

pivot_df

In [ ]:
# Plot the data
plt.figure(figsize=(15, 6))
pivot_df.plot(kind='line', stacked=False)

# Customize the plot
plt.title('Average Sentiment Over Time of Users mentioning an Airline')
plt.xlabel('Date')
plt.ylabel('Average Sentiment')
plt.legend(title='Airline', loc='upper left')
plt.grid(True)

plt.savefig('plot1_linechart.png')

# Show the plot
#plt.tight_layout()
plt.show()


In [ ]:
path_to_db = r"C:\Users\20230895\OneDrive - TU Eindhoven\Desktop\Data Science BsC\Year 1\Quartile 4\JBG030_DBL0\DBL0_easyjet\tweets4.db"

# Connect to the database
conn = sqlite3.connect(path_to_db)
cursor = conn.cursor()

# Query to fetch the necessary data
query = """
SELECT full_text, sent_label_vader
FROM tweets, extended_tweets
WHERE tweets.id_str = extended_tweets.id_str
"""
df = pd.read_sql_query(query, conn)

# Close the connection
conn.close()

# Define a function to extract airline names from tweets
def extract_airline(text):
    if '@easyJet' in text:
        return 'easyJet'
    elif '@Ryanair' in text:
        return 'Ryanair'
    elif '@British_Airways' in text or '@BritishAirways' in text:
        return 'British Airways'
    elif '@AmericanAir' in text:
        return 'AmericanAir'
    elif '@VirginAtlantic' in text:
        return 'VirginAtlantic'
    else:
        return 'Other'

# Apply the function to the dataframe
df['airline'] = df['full_text'].apply(extract_airline)

# Group by airline and sentiment to count occurrences
df_dist_sentiment = df.groupby(['airline', 'sent_label_vader']).size().unstack(fill_value=0)

# Initialize the axis for the plot
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(14,7))

# Plot the data
bars_grouped = df_dist_sentiment.plot(kind='bar', ax=ax, color=['#E74C3C', '#AAB7B8', '#2ECC71'])

# Title
ax.text(x=0.12, y=.93, s="Distribution of Sentiment per Airline", transform=fig.transFigure, ha='left', fontsize=24, weight='bold', alpha=.6)

# Axis formatting
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_color('#DDDDDD')
ax.set_axisbelow(True)
ax.yaxis.grid(True, color='#EEEEEE')
ax.xaxis.grid(False)

# X-axis
ax.set_xlabel('Airline Name')
plt.xticks(rotation=45, size=14)

# Y-axis
ax.set_ylabel('Tweets Count', size=14)

# Code for saving
fig.savefig('dist_of_sent_per_airline.png')

plt.show()
